In [1]:
%websocketconnect --password wpass ws://10.0.35.134:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


## Micropython on a light bulb

This requires bitbanging as shown in https://github.com/xoseperez/my92xx
with teardown at https://tinkerman.cat/post/ailight-hackable-rgbw-light-bulb

Before building the Micropython, the following modules should be copied over:
```
cp micropython-mqtt/mqtt_as/mqtt_as.py micropython-fork/ports/esp8266/modules/
cp micropython-mqtt/mqtt_as/mqtt_as_timeout.py micropython-fork/ports/esp8266/modules/
cp -r micropython-lib/uasyncio/uasyncio/ micropython-fork/ports/esp8266/modules/
cp micropython-lib/uasyncio.core/uasyncio/core.py micropython-fork/ports/esp8266/modules/uasyncio/
cp micropython-lib/uasyncio.queues/uasyncio/queues.py micropython-fork/ports/esp8266/modules/uasyncio/
cp -r micropython-lib/umqtt.simple/umqtt/ micropython-fork/ports/esp8266/modules/
cp micropython-lib/umqtt.simple/umqtt/simple.py micropython-fork/ports/esp8266/modules/umqtt/
cp micropython-lib/umqtt.robust/umqtt/robust.py micropython-fork/ports/esp8266/modules/umqtt/

follow instructions on http://akshaim.github.io/IoT/MicroPython_ESP8266/MP_ESP_101.html

export PATH=/home/julian/extrepository/esp-open-sdk/xtensa-lx106-elf/bin:$PATH
```

The code used for the function esp_my9291_write() is at the bottom, along with a backup of the arduino function also used.

The microcontroller is actually an ESP8285, so the flashing command is without the "-fm dio":

> esptool.py --port /dev/ttyUSB2 --baud 460800 write_flash --flash_size=detect 0 /home/julian/extrepositories/micropython-fork/ports/esp8266/build-GENERIC/firmware-combined.bin




In [ ]:
export PATH=/home/julian/extrepository/esp-open-sdk/xtensa-lx106-elf/bin:$PATH

In [2]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
b'sl\x00l\x9c\x9e|\x00\x84l\xe0|\x03\x0c\x0c\x0c\x8c'

b'd\xe4'b'c<\x87\x82\x03\xec\x13\x93{\x9bc\x8c\x0cc\x8c\xf3no\x9flog\xdc\xe3\xe4'b'b\x1cp\xe4\x87l{d{d8\xf3n\xe0\x10\x03\x0c\x0c\x82\x0cl'b'\x0c\x0cc\x0cg\xe3|\x03\xe4$l\x87\x1c\x80\x0cc\x84\xf3oo\xee\x00l\x8c\xc7l`\x03\x90\x1b\x12gg'b'd`\x03\x0f\x03o;\x87\x92\x93g'b'c\xecl`\x02x\xf3g\xe0\x10\x03'b'{\x8c\x9c\xdc\xe3\xe0'b'c\x0cg\xe3|\x03d'b'\x84'b'\x0cc\x8c\xf3gn\xe7\x00'b'l`\x02\x98\x13\x13nn'b'\x8f\x0el\x07\x02gs\xc7\x93\x93o'b'\x0c\x1bl\xc7\x0f$p\xf3o\xe0\x10\x03\x0c\x0cs\x8c\x9c\x9c\xe3\xe0'b"\x87\x0c\x0cc\x0c'\xe3|\x03"b'\x8c\x0c\x87c\x8c\xf3nn\xe7\x00\x0c\x87l`\x03\x98\x12\x13gn'b'\x8f\x07l\x0f\x02gs\x87\x93\x93n'b'\x1b\x8c\x92`\x03\x07s\x9b\x9bo\x0c\x0c\x12\x8c\x93`\x03\x84\xe3\xf3\x9bx\x03\x83og\x80l\x1bld'b'\x03l\x87c\xec\x87\x03'b'l`\x02sl\x00l\x9c\x9e|\x00\x84l\xe0|\x03\x0c\x0c\x0c\x8c'b'd\xe4'b'c<\x87\x82\x03\xec\x13\x9b{\x9bc\x8c\x0cc\x8c\xf3no\x9flog\xdc\xe3\xe4'b'b\x1cp\xe4\x87l{d{dp\xf3n\xe0\x10\x03\x0c\x0c\x82\x0cd'b'\x0c\x0c\x0cc\x0cg\xe3|\x03\xe4ll\x87\x1c\x80\x0cc\

b'\x84'b'$\xec\x0cc|\x87\x83\x03\xe4\x1b\x9b{\xdbb\x8c\x0c#\x84\xfboo\x9flgg\x9c\xe3\xec'b'c\x1cp\xec\x87l{drlp\xf2g\xe0\x18\x03'b'\x83'lcb'o\xe2|\x02\xecld\x8f\x1c\x80'b'c\x8c\xf2gg\xef\x00d\x84\x8fl`\x02\xd8\x13\x1boo\x0cl`\x02\x07\x03ns\x8f\x9b\x9bo\x0c\x0cc\xecd`\x03p\xfbo\xe0\x18\x02\x0c\x0cs\x84\x9c\x9c\xe2\xe0\x0c\x0c\x0c\x0c#'b'o\xe3|\x03l\x0c\x8c\x0c'b'b\x84\xfboo\xef\x00\x0c\x0cd`\x03\x90\x1b\x1bog\x0c\x8e\x0fl\x0e\x03o{\x8f\xdb\x92g\x0c'b'\x13d\x8f\x07lx\xf2g\xe0\x18\x03'b'{\x8c\x9c\x9c\xe3\xe0\x0c\x8e'b'c\x0co\xe2|\x02\x0c\x8c'b'\x8fc\x8c\xfbgg\xef\x00\x0c\x8fd`\x03\x90\x1b\x1bog\x0c\xc7\x0f$\x0e\x03o{\x8f\x9b\xdbg\x0c\x0c\x13\x84\x9b`\x03\x0f;\x92\x93g'b"\x1b\x84\x9b`\x02\x84\xe3\xf3\xdbx\x03\x83o'\x80l\x1bld"b'\x03l\x87c\xec\x87\x03'b'l`\x02sd\x00l\x9c\x9e|\x00\x84l\xe0|\x03\x0c\x0c\x0c\x84'b'd\xe4\x0cc|\x87\x82\x03\xe4\x13\x9b{\x9b#\x8c\x0cc\x84\xf3no\x9flgg\x9c\xe3\xec'b'b\x1cp\xec\x87l{d{$p\xf3o\xe0\x10\x03\x0c\x0c\x83\x0cd'b"\x0c\x0c\x0cc\x0c'\xe3|\x03\xe4ll\x87\

Ready.


In [9]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinDI        13
pinDCK       15
connection1  BV6000,bubblino,192.168.43.1


Sent 6 lines (149 bytes) to config.txt.


In [10]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 76 lines (2732 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [45]:
%websocketconnect --password wpass ws://10.0.35.134:8266/

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [46]:
%ls

Listing directory '/'.
      228    boot.py
      149    config.txt
     2992    main.py
     2732    stdmqttas.py
      969    utils.py
       15    webrepl_cfg.py


In [47]:
%sendtofile main.py

from mqtt_as import MQTTClient
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
import uasyncio as asyncio
from stdmqttas import shortmac, topicstem, topicstatus
from machine import Pin
from esp import my9291_write
import time, itertools

pinDI = Pin(int(fconfig["pinDI"]), Pin.OUT) 
pinDCK = Pin(int(fconfig["pinDCK"]), Pin.OUT)
my9291_write(pinDI, pinDCK, b'\x00')  # 16 bits

topicrgbwhex = topicstem+"/rgbwhex"
topiccmd = (topicstem+"/cmd").encode()
topicreply = topicstem+"/reply"
topicbeat = topicstem+"/beat"

rgbw = bytearray(8)

def conv8hex(h):
    for i in range(8):
        rgbw[i] = int(h[i*2:i*2+2], 16)
        
def conv4hex(h):
    for i in range(4):
        rgbw[i*2] = int(h[i*2:i*2+2], 16)
        rgbw[i*2+1] = 0

def conv3hex(h):
    for i in range(3):
        rgbw[i*2] = int(h[i*2:i*2+2], 16)
        rgbw[i*2+1] = 0
    rgbw[6] = 0
    rgbw[7] = 0

def RGBWhex(rgbwhex):
    try:
        if len(rgbwhex) == 16:
            conv8hex(rgbwhex)
        elif len(rgbwhex) == 8:
            conv4hex(rgbwhex)
        elif len(rgbwhex) == 7 and rgbwhex[0] == ord("#"):
            conv3hex(rgbwhex[1:])
        elif len(rgbwhex) == 6:
            conv3hex(rgbwhex)
        else:
            return False
    except ValueError as e:
        return False
    except IndexError as e:
        return False
    print("my9291_write", rgbw)
    my9291_write(pinDI, pinDCK, rgbw)
    return True


def errflash():
    my9291_write(pinDI, pinDCK, b'\x09\x00\x00\x00\x00\x00\x00\x00')
    time.sleep_ms(200)
    my9291_write(pinDI, pinDCK, rgbw)
    
async def heartbeatofflineflashes(client):
    for j in itertools.count():
        while not client.isconnected():
            await asyncio.sleep_ms(1000)
            RGBWhex("00000500")
            await asyncio.sleep_ms(1000)
            RGBWhex("00000005")

        for i in range(8):
            RGBWhex("01000000" if (i%2) else "00100000")
            await asyncio.sleep_ms(100)
        
        for i in itertools.count():
            await asyncio.sleep_ms(1000)
            if not client.isconnected():
                break
            await client.publish(topicbeat, "%d-%d"%(j, i))

def callbackcmd(topic, msg, retained):
    print("callbackcmd", topic, msg)
    if topic == topiccmd:
        aloop.create_task(callbackcmdtask(client, topicreply, msg))
    else:
        if not RGBWhex(msg):
            print("error")
            errflash()

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd)
    await client.subscribe(topicrgbwhex)
    print("subscribed to", topicrgbwhex)
            
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
client = MQTTClient(config)
client.DEBUG = True

print(topicstem)
aloop = asyncio.get_event_loop()
aloop.create_task(heartbeatofflineflashes(client))
aloop.create_task(mqttconnecttask(client))
aloop.run_forever()


Sent 105 lines (2991 bytes) to main.py.
